#**Práctica 6a: Leyendo algunos tipos de datos e introducción Keras y Tensorflow**

Curso: Inteligencia Artificial para Ingenieros

Prof. Carlos Toro N. (carlos.toro.ing@gmail.com)

2022

**Introducción**

En esta práctica leeremos algunos tipos de datos típicos con los que podemos trabajar (no usaremos texto ya que tiene muchos más desafíos) e introduciremos algunos de los conceptos básicos Tensorflow y Keras.

**PREVIO:** Importaciones necesarios para la práctica

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds


import matplotlib.pyplot as plt
import numpy as np
import os
import PIL # Pillow es otra librería para temas de procesamiento de imágenes
import PIL.Image

Antes de ejecutar las siguientes celdas, subir las imágenes y audio entregados (u otros propios)

In [ ]:
# versión actual de tensorflow
print(tf.__version__)

## **1. Leyendo otros tipos de datos**

###**1.1 Imágenes**

In [ ]:
img_names = tf.data.Dataset.list_files('*.jpg')# útil para crear/leer un dataset en base a la extensión de archivos indicado

In [ ]:
im = []
N  =len(img_names)
for i,image in enumerate(img_names):
    raw = tf.io.read_file(image)#qué tipo de variable será raw?
    im.append(tf.image.decode_jpeg(raw))
    plt.subplot(1,N,i+1)
    plt.imshow(im[i])
    plt.show

# tipo de dato de las imágenes
print(f"Las imágenes son de tipo: {type(im[0])}")

Algunas operaciones osbre las imágenes

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.title('Ajuste de brillo')
plt.imshow(tf.image.adjust_brightness(im[1],delta = 0.3))#ajuste del brillo, el parámetro toma valores entre  0 y 1, 1 para blanco

plt.subplot(1,2,2)
plt.title('Refleja imagen horizontalmente')
plt.imshow(tf.image.flip_left_right(im[1]))#refleja horizontalmente la imagen
plt.show()

###**1.2 Audio**

Uasremos en este caso la libreria ``librosa`` de python, dedicada para trabajar con música y audio.

In [ ]:
#Algunos imports necesarios
import librosa
import librosa.display as dsp
from IPython.display import Audio
import matplotlib.pyplot as plt

Cargaremos un par de audios disponibles en la librería, uno de sonido monocanal, y uno stereo.

In [ ]:
# archivo mono canal
audio1, Fs1 = librosa.load(librosa.util.example_audio_file(),duration = 60)
Audio(data = audio1, rate = Fs1)

Información del audio:

In [ ]:
print('Forma de los datos',audio1.shape)# tenemos una dimension de dos columnas, porque el audio es de dos canales
print('Frecuencia de muestreo: ',Fs1,'Hz')
print('Número de muestras totales: ', audio1.shape)

In [ ]:
# archivo de audio stereo
audio2, Fs2 = librosa.load(librosa.util.example_audio_file(),mono = False,duration = 60)
Audio(data = audio2, rate = Fs2)

Información del audio:

In [ ]:
print('Forma de los datos',audio2.shape)# tenemos una dimension de dos filas, porque el audio es de dos canales ahora
print('Frecuencia de muestreo: ',Fs2,'Hz')
print('Número de muestras totales: ', audio2.shape[1])

En este caso analicemos el audio mono canal:

In [ ]:
plt.figure(figsize = (10,3))
librosa.display.waveshow(audio1, sr=Fs1)
plt.title('Audio Mono')
plt.ylabel('Amplitud')
plt.xlabel('Tiempo')
plt.show()

Representación de la señal en tiempo-frecuencia con la STFT,
utilizamos la función de ``stft()`` de la libreria, la documentación la pueden encontrar [acá](http://librosa.org/doc/main/generated/librosa.stft.html).

In [ ]:
d = librosa.stft(audio1)# calcula la Transformada de Fourier de Tiempo Corto (Short-Time Fourier Transform)
D = librosa.amplitude_to_db(np.abs(d),ref=np.max)# pasamos a db

plt.figure(figsize = (10,3))
dsp.specshow(D, y_axis='linear', x_axis='s',sr=Fs1)
plt.show()

##**2. Dataset Augmentation**

En este ejemplo usaremos el módulo ``tfds`` entrega un conjunto de datasets para usar con tensorflow y otros frameworks de Machine Learning.

El ejemplo siguiente fue tomado desde [documentación oficial tensorflow](https://www.tensorflow.org/tutorials/images/data_augmentation)

In [ ]:
# Descargamos un dataset
(train_ds, val_ds, test_ds), info = tfds.load(
                                                'tf_flowers',
                                                split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
                                                with_info=True,
                                                as_supervised=True,
                                              )

In [ ]:
# este dataset contiene 5 clases
print('Número de clases: ',info.features['label'].num_classes)
print('Nombre de clases: ',info.features['label'].names)
print('Número total de imágenes de entrenamiento: ',info.splits['train'].num_examples)

In [ ]:
# mostramos algunas imagenes disponibles
get_label_name = info.features['label'].int2str
data_iter    = iter(train_ds)# iter genera un iterador sobre el objeto que hace referencia al dataset

plt.figure(figsize=(10, 10))
for i in range(9):
  image, label = next(data_iter)#next va accediendo a los elementos de este iterable
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image)
  plt.title(get_label_name(label))
  plt.axis('off')

Usamos keras para realizar algunas transformaciones sobre las imágenes

In [ ]:
IMG_SIZE = 180
image, label    = next(iter(train_ds))
resize_and_rescale = keras.Sequential([
                                          layers.Resizing(IMG_SIZE, IMG_SIZE),
                                          layers.Rescaling(1./255)
                                         ])

result = resize_and_rescale(image)
plt.imshow(result)
plt.show()

# nueva escala de los valores de los pixeles
print("Min and max pixel values:", result.numpy().min(), result.numpy().max())

Aumentación del dataset completo:

In [ ]:
#definimos las transformaciones
data_augmentation = keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),
                                      layers.RandomRotation(0.2)])

In [ ]:
# Ejecutamos algunas iteraciones para llamar a data_augmentation()
# cada vez que se llame ejecutará una de las operaciones de forma aleatoria
image, label    = next(iter(train_ds))# cargamos una imagen
# Añadimos una dimensión a la imagen a transformar para usar el método definido
image = tf.expand_dims(image, 0)

plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = data_augmentation(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0].numpy().astype('uint8'))
  plt.axis("off")
plt.show()

Ejemplo tomado desde acá: https://www.tensorflow.org/tutorials/images/data_augmentation

##**3. Módulos comúnes de Tensorflow**

Tanto en los modelos de machine learning que hemos visto como los de deep learning, tenemos un ciclo de vida de los modelos, en particular, veremos como se implementa usando la API de alto nivel de Tensorflow: Keras,

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

Dos formas de construir modelos:
con el enfoque secuencial/apilado o el método funcional.

In [ ]:
modelo_apilado = tf.keras.Sequential()#modelo apilado, la forma más común de construir modelos

modelo_apilado.add(layers.Dense(32,activation = 'relu'))#Dense hace referencia a una capa fully connected
modelo_apilado.add(layers.Dense(32,activation = 'relu'))
modelo_apilado.add(layers.Dense(32,activation = 'relu'))
modelo_apilado.add(layers.Dense(10,activation = 'softmax'))


Usando modelos funcionales con `tf.keras.Input` y `tf.keras.Model` Construir modelos de esta forma es más compleja pero a su vez es más versatil
permite pasar variables y datos entre diferentes fases de procesamiento. Estos se prefieren en casos donde se requiere más de una salida.

In [ ]:
x  = tf.keras.Input(shape=(32,))
h1 = layers.Dense(32,activation='relu')(x)
h2 = layers.Dense(32,activation ='relu')(h1)
y  = layers.Dense(10,activation='softmax')(h2)

modelo_funcional = tf.keras.models.Model(x,y)
modelo_funcional.summary()

### **Capas que podemos utilizar con tf.keras.layers**

* tf.keras.layers.Dense: Crea capas completamente conectadas
* tf.keras.layers.Conv2D: crea una capa conv. bidimensional
* tf.keras.MaxPooling2D
* tf.keras.AveragePooling2D
* tf.keras.layers.RNN
* tf.keras.layers.LSTM
* tf.keras.layers.GRU
* tf.keras.Dropout

In [ ]:
help(tf.keras.layers.Dense)

In [ ]:
# dos formas de definir la función de activación en una capa
layers.Dense(32, activation = 'sigmoid')
layers.Dense(32, activation = tf.sigmoid)

In [ ]:
# Regularización
layers.Dense(32,kernel_regularizer=tf.keras.regularizers.l2())

###**Entrenamiento y Evaluación**

Luego de que los modelos son definidos, compile() se llama para configurar el proceso de entrenamiento del modelo.

Aquí podemos definir términos como:

* tipo de optimizador: ej. rmsprop (default), adam, ...
* loss function: ej. cross entropy para tareas de clasificación, MSE para tareas de regresión.
* metrics: ej. accuracy
* loss_weights

In [ ]:
model2 = tf.keras.Sequential()#definimos un modelo secuencial
help(model2.compile)

In [ ]:
# por ejemplo:
model2.add(layers.Dense(10,activation='softmax'))
model2.compile(optimizer='Adam',loss=tf.keras.losses.categorical_crossentropy,metrics=[tf.keras.metrics.categorical_accuracy])

Luego, para entrenar el modelo podemos ocupar el método fit, este recibe como argumentos, entre otros:

* los datos de entrenamiento
* el número de epocas que se entrenará el modelo, recordar, cada época representa una pasada completa del set de datos por la red.
* el tamaño del batch que estará viendo la red antes de actualizar los pesos,
* también se pueden incluir datos de validación para aplicar estrategias como detención temprana (early stopping)

In [ ]:
# simulemos algunos datos y entrenemos el modelo definido
import numpy as np

train_x = np.random.random((1000,36))
train_y = np.random.random((1000,10))

val_x = np.random.random((200,36))
val_y = np.random.random((200,10))


model2.fit(train_x,train_y,epochs=10,batch_size=100,validation_data=(val_x,val_y))

### **Funciones de Callback**
Estas será útiles para configurar otras  propiedades del entrenamiento, por ejemplo:

* guardar periódicamente los modelos
* cambiar dinámicamente la tasa de aprendizaje
* configurar la estrategia de early stopping
* usar TensorBoard para visualizar el proceso de entrenamiento

In [ ]:
import os
# Load the TensorBoard notebook extension
%load_ext tensorboard


logdir=os.path.join("logs")# para guardar el registro del entrenamiento y visualizarlo luego con TensorBoard
if not os.path.exists(logdir):
    os.mkdir(logdir)

#Número de epocas
Epochs = 10

# diseñamos una estrategia para cambiar dinámicamente la tasa de aprendizaje durante el entrenamiento
# ojo, que algunos optimizadores ya lo hacen, pero esto es mas personalizado
def lr_Scheduler(epoch):
    if epoch > 0.9*Epochs:
        lr=0.0001
    elif epoch > 0.5*Epochs:
        lr = 0.001
    elif epoch > 0.25*Epochs:
        lr=0.01
    else:
        lr=0.1

    print(lr)
    return lr


callbacks = [
    #Early stopping:
    tf.keras.callbacks.EarlyStopping(
        #Métrica para deterinar si el modelo aún se puede mejorar más o no
        monitor='val_loss',
        #cantidad de cambio minima sobre la metrica usada para evaluar si hubo cambio en el desempeño del modelo
        min_delta=1e-2,
        # Número de épocas en las cuales el desempeño del modelo no ha mejorado su desempeño
        patience=2),
    #Guardamos periódicamente el modelo:
    tf.keras.callbacks.ModelCheckpoint(
        #filepath y nombre del modelo a guardar
        filepath='testmodel.h5',
        #en caso de querer guardar solo el mejor modelo
        save_best_only=True,
        #Métrica a monitorear
        monitor='val_loss'),
    #Cambio dinámico en la tasa de aprendizaje con el scheduler definido por nosotros.
    tf.keras.callbacks.LearningRateScheduler(lr_Scheduler),
    #Uso de TensorBoard.
    tf.keras.callbacks.TensorBoard(log_dir=logdir)
]


In [ ]:
model2.fit(train_x,train_y,batch_size=15, epochs=Epochs,callbacks=callbacks,validation_data=(val_x,val_y))

In [ ]:
# desplegamos resultados en dashboard TensorBoard, opcional. Más detalles en: https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/get_started.ipynb#scrollTo=6B95Hb6YVgPZ
%tensorboard --logdir logs/train

### **Evaluación y predicción usando el modelo entrenado**

Usaremos en este caso los métodos:
* `.evaluate()`: predice la salida para una determinada entrada y luego calcula la(s) métrica(s) especificadas en .compile() y basado en los valores y_true e y_pred retorna la métrica como valor de salida.
* `.predict()`: solo retorna y_pred

respectivamente

In [ ]:
test_x = np.random.random((1000,36))
test_y = np.random.random((1000,10))

print('Evaluación en datos de prueba:\n')
results = model2.evaluate(test_x,test_y)
print("Test loss, test acc: ", results)

# predicciones en datos
print('\nGeneramos predicciones para 3 muestras:\n')
prediccion = model2.predict(test_x[:4])
print("Forma de la predicción: ", prediccion.shape)

### **Guardar y cargar el modelo entrenado**

Podemos usar los métodos:
* TensorFlow SavedModel format: recomendado
* Keras H5 format: más antiguo con menos información

mas información acerca de las formas de guardar modelos [acá](https://www.tensorflow.org/api_docs/python/tf/keras/Model#save)

In [ ]:
from tensorflow import keras
# llamando a `save('my_model')` crea un directorio `my_model` donde se guarda el modelo
model2.save("my_model")# por defecto guarda en formato tf, si escriben my_model.h5, guardarán en foromato .h5

# Puede ser usado para reconstruir el modelo igualmente
reconstructed_model = keras.models.load_model("my_model")

In [ ]:
# predicciones en datos usando el modelo cargado
print('\nGeneramos predicciones para 1 muestra:\n')
prediccion = reconstructed_model.predict(test_x[[100],:])# muestra 100
print(prediccion)
print("suma de las probabilidades obtenidas:", prediccion.sum())

### **Otra información que podemos acceder desde los modelos entrenados**

1. Acceder a los valores de los pesos dentro de cada capa de la red neuronal, usemos para esto el modelo entrenado en la sección anterior, el cual consistía de un modelo de red completamento conectado, con una sola capa:  `model2.add(layers.Dense(10,activation='softmax'))`

In [ ]:
#Inspeccionemos el modelo
model2.summary()

Son 370 parámetros que debieramos haber entrenado.

In [ ]:
for layerNum, layer in enumerate(model2.layers):
    weights = layer.get_weights()[0]
    biases = layer.get_weights()[1]
    print(np.shape(weights))
    print(np.shape(biases))

In [ ]:
print('Pesos de la primera neurona:\n',weights[:,0])
print('Bias de la primera neurona: ',biases[0])

**Ejercicios:**

1. Existen otras transformaciones que se pueden incluir, entre estas: layers.RandomContrast, layers.RandomCrop,layers.RandomZoom, implementarlas en el set de datos anterior y graficar algunos ejemplos.

2. Explorar otros datasets disponibles en Tensorflow

3. ¿Qué argumentos en las definiciones de las capas, metodos para configurar el entrenamiento y método para realizar el entrenamiento vistos son obligatorios y cuales son opcionales?, de los opcionales, ¿qué valores por defecto asumen?

4. Desafío: Con lo que ya hemos visto, implementar una red neuronal de una capa oculta para un problema de regresión o clasificación vistos en clases anteriores. Tenga cuidado con las dimensiones y definción de capa salida.